# Multi-Client Example
*BeamNG.tech / BeamNG.drive*

## Introduction

This example shows how to connect multiple `BeamNGpy` instances to the simulator and have them control different vehicles. For demonstration purposes, both instances are housed in the same Python process, but the same example could be used from an entirely different process or machine as well.

## Scenario

The scenario will be a rather simple one: there are two vehicles on the `smallgrid` map (an infinite flat plane) and one vehicle will randomly drive around, controlled by client A, with another vehicle, controlled by client B, mimicking A's behavior.

## Setup

Contrary to other examples, we will be using two instances of the `BeamNGpy` class representing two clients A and B. In our case, client A will create the scenario containing both vehicles, but client B will later connect and control one of the vehicles while A controls the other. The classes involved in this are:

* `BeamNGpy`: Two instances of this will be used to implement client A and B
* `Scenario`: Client A will create a scenario for both clients to play in
* `Vehicle`: Used to specify and control vehicles involved in the scenario
* `Electrics`: A sensor used to inspect interal systems of the vehicle

The code starts with importing the respective classes:

In [1]:
import random

import numpy as np

from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Electrics

Now we set up our first client who will create the scenario for both clients. Client A will also launch the simulator.

In [2]:
client_a = BeamNGpy('localhost', 64256)
client_a.open()

With the simulator running, we can start setting up our scenario. It will contain two vehicles, the one controlled by Client A being placed in front of the one later controlled by client B.

In [3]:
scenario = Scenario('smallgrid', 'tag')
av_a = Vehicle('vehicleA', model='etk800')
av_b = Vehicle('vehicleB', model='etk800')
scenario.add_vehicle(av_a, pos=(0, -10, 0))
scenario.add_vehicle(av_b)
scenario.make(client_a)

## Running

The scenario is now made, meaning the required files have been generated and can be loaded in the simulator:

In [4]:
client_a.load_scenario(scenario)
client_a.start_scenario()

Now we will set up our second client and connect it to the running simulator. The client will first connect, then query the running scenario, and retrieve currently active vehicles. They will then find the vehicle meant for Client B and connect to it. Note that `client_b` is opened with the flag `launch=False`meaning it will not launch its own BeamNG.tech process.

In [5]:
client_b = BeamNGpy('localhost', 64256)
client_b.open(launch=False)
running_scenario = client_b.get_current_scenario()
print(running_scenario.name)
active_vehicles = client_b.get_current_vehicles()
bv_a = active_vehicles['vehicleA']
bv_b = active_vehicles['vehicleB']
# B attaches their own sensor to get the current controls of A
bv_a.attach_sensor('electrics', Electrics())
bv_a.connect(client_b)
bv_b.connect(client_b)

tag


Two clients are now connected to the running simulation and both vehicles. What follows is the main loop of the scenario, where Client A sends random steering inputs to their vehicle and Client B checks how A's vehicle is driving using the electrics sensor and sends the same inputs to their vehicle.

In [6]:
# Focus simulator on second vehicle because it's the more interesting one to see
client_a.switch_vehicle(av_b)

for _ in range(120):
    # Client A sending control inputs to their vehicle connection
    av_a.control(throttle=random.random(), steering=(random.random() * 2 - 1))
    
    # Client B updating the electrics sensor of A's vehicle
    bv_a.poll_sensors()
    
    throttle = bv_a.sensors['electrics'].data['throttle_input']
    steering = bv_a.sensors['electrics'].data['steering_input']
    print(throttle, steering)
    bv_b.control(throttle=throttle, steering=steering)
    
    # Client A now advancing the simulation 60 steps
    client_a.step(60)

client_b.close()
client_a.close()

0.7577024890931076 -0.029248118614031222
0.41466409511715085 -0.6133109369707077
0.6612916857332573 0.7583303629890917
0.7297772057663137 -0.09513516033971536
0.4971688804966732 -0.3535049334656909
0.815687162804359 0.19232261777649018
0.5468545547028463 -0.1681908719065344
0.18177021312567299 -0.8860084429572372
0.7972282046989743 0.6561286825906633
0.9244512533033096 0.8401853896253498
0.9297621871157442 0.480516448173214
0.7268450870950331 0.4628886928545947
0.05240987322725532 -0.24389971202574562
0.06647287793522028 -0.09850204721673225
0.89240753173758 0.7633874594184644
0.22430070428904925 -0.5108735598848688
0.9514956002747675 0.3363851117363099
0.8950055623685286 0.8726946354699725
0.8547668591694162 0.02054363416370171
0.2528524505451656 0.22841815749675592
0.46332827386756315 0.3021335200854024
0.44573713232987433 -0.8422713505738366
0.22223019808146538 -0.6579719552737314
0.7933353969372042 0.6012680472427994
0.6167286727885309 -0.7458466561741002
0.4106512930647186 -0.0505